In [1]:
# Import necessary libraries
import numpy as np
import scipy.sparse as sp
import scipy.stats as ss
import matplotlib.pyplot as plt
import math

num_edge_features = 5
num_node_features = 3
connect_prob = 0.1

In [2]:
# Lists of ISO standard modules and tooth numbers

modules = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.8, 1, 1.25, 1.5, 2, 2.5, 3, 4, 5, 6, 8, 10, 12, 16, 20, 25, 32, 40, 50,
           0.15, 0.25, 0.35, 0.45, 0.55, 0.7, 0.75, 0.9, 1.75, 2.25, 2.75, 3.5, 4.5, 5.5, 7, 9, 11, 14, 18, 22, 28, 36, 45,
           0.65, 3.25, 3.75, 6.5]
teeth = [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 
         41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
         70, 72, 75, 76, 80, 85, 90, 95, 100, 110, 114, 120, 127]

In [65]:
# Import necessary libraries
import numpy as np
import scipy.sparse as sp
import scipy.stats as ss
import math as math

ef = 5
nf = 3
connect_prob = 0.1

# Lists of ISO standard modules and tooth numbers

modules = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.8, 1, 1.25, 1.5, 2, 2.5, 3, 4, 5, 6, 8, 10, 12, 16, 20, 25, 32, 40, 50,
           0.15, 0.25, 0.35, 0.45, 0.55, 0.7, 0.75, 0.9, 1.75, 2.25, 2.75, 3.5, 4.5, 5.5, 7, 9, 11, 14, 18, 22, 28, 36, 45,
           0.65, 3.25, 3.75, 6.5]
teeth = [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 
         41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
         70, 72, 75, 76, 80, 85, 90, 95, 100, 110, 114, 120, 127]

# Choose int in a range from a normal distribution
def normRange(low, high):
    x = np.arange(low, high+1)
    xU, xL = x + 0.5, x - 0.5 
    prob = ss.norm.cdf(xU, loc = ((low+high)/2), scale = ((high-low)/5)) - ss.norm.cdf(xL, loc = ((low+high)/2), scale = ((high-low)/6))
    prob = prob / prob.sum() # normalize the probabilities so their sum is 1
    
    return np.random.choice(x, p = prob)





In [11]:
np.where(A[[6,7],:,0])

(array([0, 1]), array([5, 3]))

In [11]:
def check_redundant(A,i):
    v_ne = list(np.where(A[i,:,0])[0])
    v_ne = list(set(v_ne))
    
    n_ne = len(v_ne)
    
    while True:
        if 0 in v_ne and A.shape[0]-1 in v_ne:
            return False
        else:
            v_ne += list(np.where(A[v_ne,:,0])[1])
            v_ne = list(set(v_ne))
            
        if len(v_ne) == n_ne:
            return True
        else:
            n_ne = len(v_ne)

In [34]:
def check_locking(A):
        
    return False

In [73]:
def generate_random_gear_box(n=None, Max_N = 10, Min_N = 3, C_prob = 0.1):
    if n is None:
        n = np.random.randint(low=Min_N,high=Max_N+1)

    # Instantiate adjacency matrix  
    A = np.zeros((n, n, 6))

    # Loop through the edges in the graph and assign their features
    for i in range(n):
        connected = False
        while not connected:
            for j in range(n):
                if (i != j) and (np.random.uniform() <= connect_prob) and not (i==0 and j==n-1) and not (j==0 and i==n-1):
                    m = np.random.choice(modules) # module in mm
                    t1 = np.random.choice(teeth) # tooth of gear 1
                    t2 = np.random.choice(teeth) # tooth of gear 2
                    alpha = math.pi/9 # pressure angle in radians
                    fw = (m*(t1+t2))/2 # face width in mm

                    A[i,j,0] = 1
                    A[i,j,1] = m
                    A[i,j,2] = t1
                    A[i,j,3] = t2
                    A[i,j,4] = alpha
                    A[i,j,5] = fw
                    connected = True
    
        # Loop through the nodes in the graph and assign their features
    x0 = 0
    y0 = 0
    for i in range(1,n-1):
        # Set origin to input shaft
        if i == 0:
            x = 0
            y = 0
            theta = 0
        else:
            #loop until there's a connection, and get relative position
            for j in range(1,n-1):
                if A[i-1,j-1,0] == 1:
                    ctc = A[i,j,1]*(A[i,j,2]+A[i,j,3]) # center to center distance = m*(t1+t2)
                    theta = np.random.uniform(2*math.pi)
                    x = x0 + ctc * math.sin(theta)
                    y = y0 + ctc * math.cos(theta)
        x0 = x
        y0 = y

    redundant_shafts = []
    for i in range(1,n-1):
        if check_redundant(A,i):
            redundant_shafts.append(i)
            
    non_redundant = np.arange(n)
    non_redundant = non_redundant[np.logical_not(np.isin(non_redundant,redundant_shafts))]
    
    A = A[non_redundant,:,:][:,non_redundant,:]
    
    #if check_locking(A):
    #    return generate_random_gear_box(n, Max_N, Min_N, C_prob)
    
    # Instantiate an empty node feature matrix with the desired shape
    N = np.zeros((n, nf))


    return A

In [84]:


A = generate_random_gear_box(8)

IndexError: index 2 is out of bounds for axis 1 with size 2

In [62]:
A[:,:,0].astype(int)

array([[0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0],
       [1, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [1, 1, 1, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0]])